# secrets

In [16]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime as dt
import pandas as pd
import numpy as np
import pprint
import boto3
import json
import time
import math
import sys
import os

os.chdir(path = '/home/steven/Documents/reddit-streaming/')
base = os.getcwd()
pp = pprint.PrettyPrinter(indent = 1)
print("imported modules, base: {}.".format(base))

imported modules, base: /home/steven/Documents/reddit-streaming.


## read creds json

In [7]:
with open("creds.json", "r") as f:
    creds = json.load(f)
    f.close()

# pp.pprint(creds)

## spark

In [15]:
spark_host = "spark" 
kafka_host = "kafka" 
subreddit = "cosmos"
aws_client = creds["aws_client"]
aws_secret = creds["aws_secret"]

spark = SparkSession.builder.appName("cosmos") \
    .master("spark://{}:7077".format(spark_host)) \
    .config("spark.scheduler.mode", "FAIR") \
    .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
    .config("spark.executor.memory", "4096m") \
    .config("spark.executor.cores", "4") \
    .config("spark.streaming.concurrentJobs", "4") \
    .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
    .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:///opt/workspace/events/{}/".format(subreddit)) \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.hadoop:hadoop-common:3.3.1,org.apache.hadoop:hadoop-aws:3.3.1,org.apache.hadoop:hadoop-client:3.3.1,io.delta:delta-core_2.12:1.2.1") \
    .config("spark.hadoop.fs.s3a.access.key", aws_client) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config('spark.hadoop.fs.s3a.buffer.dir', '/opt/workspace/tmp/blocks') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.SparkContext()
print("created spark successfully.")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/steven/.ivy2/cache
The jars for the packages stored in: /home/steven/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aaae753b-1e9d-48dd-a6c9-4da824d43bf9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.30 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in local-m2-cache
	found org.spark-project.spark#unused;1.0.0 in local-m2-ca

RuntimeError: Java gateway process exited before sending its port number